In [23]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests

In [2]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [C:\Users\alexz\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


# Step 1 - Scraping

### NASA Mars News - Print Article Title & Description

In [3]:
# Visit the NASA Mars News Site
url_news = "https://redplanetscience.com/"
browser.visit(url_news)

In [4]:
#Put results into HTML
html = browser.html

#use BeautifulSoup to print the results
soup = bs(html, 'html.parser')
# print(soup.prettify()) - commenting out for sake of scrolling

In [5]:
# Scrape the latest news title
news_title = browser.find_by_css('.content_title')[0].text
print(news_title)

Media Get a Close-Up of NASA's Mars 2020 Rover


In [6]:
# Scrape the Latest Paragraph Text
news_paragraph = browser.find_by_css('.article_teaser_body')[0].text
print(news_paragraph)

The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch.


### JPL Mars Space Images -  Print Featured Image

In [7]:
# Visit https://spaceimages-mars.com/
url_pic = "https://spaceimages-mars.com/"
browser.visit(url_pic)

In [8]:
#Put results into html
space_image = browser.html

#Use BeautifulSoup to print the results
soup_image = bs(space_image, 'html.parser')
#print(soup_image.prettify()) - commenting out for sake of scrolling

In [9]:
#Went through elements and saw the class of the featured image was headerimage fade-in
#code below searches for the class and prints the image with
featured_pic = soup_image.find(class_='headerimage fade-in')['src']
print(featured_pic)

image/featured/mars2.jpg


In [10]:
#Assign the url string to a variable
featured_image_url = url_pic + featured_pic
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


### Mars Facts - Scrape Table

In [35]:
# Visit & Scrape data from https://galaxyfacts-mars.com/
url_facts = "https://galaxyfacts-mars.com/"
browser.visit(url_facts)

In [36]:
#Find/Scrape the table and convert it to a HTML string
html = browser.html
tables = pd.read_html(url_facts)
html_table = tables[0].to_html(header=False, index=False)
html_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Mars - Earth Comparison</td>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [37]:
#Display tables to confirm accuracy
tables = pd.read_html(url_facts)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [40]:
#Turn scraped data into DF
facts_df = tables[1]
facts_df.columns = ['Fact', 'Data']
facts_df

,Fact,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [79]:
# Visit the NASA Mars News Site
url_hemi = "https://marshemispheres.com/"
browser.visit(url_hemi)

In [80]:
#Put results into HTML
html = browser.html

#use BeautifulSoup to print the results
soup = bs(html, 'html.parser')
#print(soup.prettify())  - commenting out for sake of scrolling

In [81]:
image_div_items = soup.find_all('div',class_='item')

In [82]:
# Create temporary blank dicts to store urls
product_list = []
url_list = []
product_img_url_dict = []

In [83]:
# Create for loop that finds image urls for each hemisphere
for hemisphere in image_div_items:
    if hemisphere.h3:
        product = hemisphere.h3.text
        product_list.append(product)
    if hemisphere.img:
        image_src = hemisphere.find('img')['src']
        url_list.append(image_src)
        
# Create for loop to add the lists into blank dicts
for products, img_url in zip(product_list, url_list):
    product_url = {"Product": product, "image_url": url_hemi+img_url}
    product_img_url_dict.append(product_url)

In [84]:
product_img_url_dict 

[{'Product': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'Product': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'Product': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'Product': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]

In [85]:
browser.quit()